## Overview

This report details the **E**xtract **T**ransform **L**oad steps taken for this project. 4 datasets were used for this project. 3 were retrieved from [Kaggle.com](https://www.kaggle.com/datasets/pranav941/-world-food-wealth-bank?resource=download&select=crop1.csv). 1 was generator for use in this project from an external website, for which this report details the extraction processes from that external website, and why we extracted it. Also detailed are the transformation steps we took in order to clean our data for user readability and for database loading and accessession. 

## Extract

We began by creating a dataset on continental and country codes from the website [statisticstimes.com](https://statisticstimes.com/geography/countries-by-continents.php). We created this dataset for ideal location grouping and for standardized sorting that the original datasets lacked. We then created a DataFrame from this dataset which we utilize in our transformation. After, we then imported our `crops1` and `live1` .csv file as DataFrames for manipulation. One of our datasets `pop1` is loaded later before its crucial transformation and loading step.

## Transform

Starting, we had to reduce dataset filesize for GitHub compliancy. Our original `crops1.csv` is  `105.844MB`, which would be blocked. We reduced the filesize to `89.343MB` which maintained GitHub compliance. We accomplished this by removing any data prior to the date `1980` from crop1.csv, which held data from `1960-2020`. Removal of dates prior to `1980` was kept consistent across the other two datasets, `live1.csv` and `pop1.csv`. The trimmed `crop1` and `live1` are then loaded once again as DataFrames containing 6 columns each:

| Area | Item| Element | Year| Unit | Value |
|--------|-------|-----------|-------|------|---------|
|   Country of Origin  |Specific Crop/Livestock| Relative information of Item | The year| Units relative to Element | Value relative to unit| 

A seperate DataFrame for either `crops` and `livestock` containing `Item` and `Element` was created, from which a single DataFrame containing both crops and livestock was created. This DataFrame contains a single listing for every item and its corresponding element found in the main two dataframes, which have `1,372,196` entries in crops and `86,018` entries in livestock. 

In anticipation for database loading we renamed the column names to:


|New| country | item_name| item_type | year| item_unit | value |
|-|--------|-------|-----------|-------|------|---------|
|Old| Area | Item| Element | Year| Unit | Value |

Next, from the dataset we created for continental data and country codes, we recieved a DataFrame with 7 columns that we trimmed down to 4: `Country or Area`, `Iso-alpha3 Code`, `Region 1` and `Continent`. In anticipation for database loading we renamed these to: 

|New| country | country_code | region | continent|
|-|--------|-------|-----------|-------|
|Old|  Country or Area | Iso-alpha3 Code| Region 1 | Continent| 

A single listing DataFrame exclusively for years was created in anticipation for database loading. 

Finally, the `pop1.csv` dataset was loaded as a DataFrame. Null values were dropped and it was merged with countries DataFrame. A unique issue the population DataFrame had was year-wise compliance with our other two DataFrames of crops and livestock. In the population DataFrame, years were by column with country listed by row. The pandas method `.melt` was utilized to remedy this problem - which required initial setup to function properly. A list of the years was created from the single listing DataFrame for years. The list had all its integer values converted to strings. The pandas method was then able to properly function and create a DataFrame that shared dimensions with crops and livestock, and therefore share table compliance when loaded and queried from the database later. 

As with the rest, in anticipation of database loading we renamed the column names from population:

|New| country | country_code |
|-|--------|-------|
|Old|  Country | Country Code|

## Load

Finally, our data was loaded into a PostgreSQL database using `sqlalchemy`. Our schema was created to contain 6 tables:

- `tbl_item` 
- `tbl_crops` 
- `tbl_livestock` 
- `tbl_countries` 
- `tbl_population` 
- `tbl_year`

which the data was loaded into vis a vis each `tbl_` for pertinent information from our transformed DataFrames. This process is not autonomous. To update the user will have to periodically, likely on a year-to-year basis load the database with the previous years new data. The process should be effortless, given instruction is properly followed.